## 系統和路徑地確認，以及前置設定

**確認python版本以及tensorflow版本**

In [ ]:

!pip show tensorflow

import sys
print(sys.version)

Name: tensorflow
Version: 2.15.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine-rl, tf_keras
3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
Python 3.10.12


請先確認Jupyter筆記本設置是否正確，首先點選主選單的「修改」─「筆記本設置」─「運行類別」，選擇「Python3」，同時將「硬件加速器」下拉式選單由「None」改成「GPU」，再按「保存」。接著可選擇性執行下列指令確認Colaboratory提供的虛擬機的CPU, 磁碟空間、記憶體大小及GPU是否正確啟動。若出現'/device:GPU:0'表示GPU成功啟動。


In [ ]:
print("CPU Status:")
!cat /proc/cpuinfo | grep model\ name # 檢查CPU資訊

print("\nDisk Status:")
!df -lh # 檢查磁碟空間

print("\nRAM Status:")
!free -h #檢查記憶體大小

print("\nGPU Status:")
import tensorflow as tf
tf.test.gpu_device_name() #檢查GPU是否啟動，若無啟動則會自動改由CPU執行

CPU Status:
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz

Disk Status:
Filesystem      Size  Used Avail Use% Mounted on
overlay         202G   31G  171G  16% /
tmpfs            64M     0   64M   0% /dev
shm              31G     0   31G   0% /dev/shm
/dev/root       2.0G  1.1G  849M  57% /usr/sbin/docker-init
tmpfs            32G   68K   32

'/device:GPU:0'

**載入Google Drive中的路徑**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 函數定義

設定卷積神經網絡（CNN）中的操作

In [ ]:
import numpy as np
from numba import cuda
import math


def expand_data(data):
    '''
    這個函數的目的是將給定的三維數據張量（形狀為 ((x,y,z)）擴展為更大的尺寸（形狀為(x∗3,y∗3,z)）。
    這是為了準備將輸入數據與卷積核進行卷積操作，因為這裡使用了 3×3 的卷積核並且步長為 3。
    該函數會將原始數據在每個維度上擴展三倍。
    '''

    d1 = np.zeros([data.shape[0]*3, data.shape[1], data.shape[2]])

    for i in range(data.shape[0]):
        if i >= (data.shape[0] - 2):
            d1[i*3:(i*3+data.shape[0]-i),:,:] = data[i:,:,:]
        else:
            d1[i*3:(i*3)+3,:,:] = data[i:i+3,:,:]


    d2 = np.zeros([d1.shape[0], d1.shape[1]*3, d1.shape[2]])

    for j in range(d1.shape[1]):
        if j >= (d1.shape[1] - 2):
            d2[:,j*3:(j*3+d1.shape[1]-j),:] = d1[:,j:,:]
        else:
            d2[:,j*3:(j*3)+3,:] = d1[:,j:(j+3),:]
    return d2


def preprocess_kernel(data):


    '''
    這個函數對擴展後的數據進行預處理，以準備卷積操作。
    它首先創建了兩個和輸入數據相同形狀的全零數據，然後對於每個 3×3 的子區塊，根據指定的公式進行處理。
    這個公式可以理解為對子區塊應用了某種卷積核，將其乘以不同的權重，然後對所有子區塊的處理結果進行累加，
    最終得到預處理後的數據。
    '''

    data1 = np.zeros(data.shape)
    data2 = np.zeros(data.shape)

    for i in range(int(data.shape[0]/3)):
        k = data[(i*3):(i*3+3),:,:]
        data1[i*3,:,:] = 2*k[0,:,:] - k[1,:,:] - k[2,:,:]
        data1[i*3+1,:,:] = 2*k[1,:,:] - k[0,:,:] - k[2,:,:]
        data1[i*3+2,:,:] = 2*k[2,:,:] - k[0,:,:] - k[1,:,:]

    for i in range(int(data.shape[1]/3)):
        k = data[:,(i*3):(i*3+3),:]
        data1[:,i*3,:] = 2*k[:,0,:] - k[:,1,:] - k[:,2,:]
        data1[:,i*3+1,:] = 2*k[:,1,:] - k[:,0,:] - k[:,2,:]
        data1[:,i*3+2,:] = 2*k[:,2,:] - k[:,0,:] - k[:,1,:]

    return data1 + data2





def self_define_cnn_kernel_process(data):

    '''
    這是整個操作的主函數。它接受一個四維的輸入數據張量 (通常是作為CNN的一層輸入)，並對每個通道 (或稱為深度) 進行處理。
    對於每個通道，它首先調用 expand_data 對數據進行擴展，然後再調用 preprocess_kernel 進行預處理。
    最後返回處理後的數據張量
    '''



    '''
    1. expand data from (x, y, z) to (x*3, y*3, z) (Because Conv2D convolution with stride (3,3) for our preprocess)

    2. 3*3 kernel process:

        [2*V_1 - V_2 - V3
         2*V_2 - V_1 - V3
         2*V_3 - V_1 - V2]

        +

        [2*Vt_1 - Vt_2 - Vt_3, 2*Vt_2 - Vt_1 - Vt_3, 2*Vt_3 - Vt_1 - Vt_2]

    '''
    #input
    data_final = np.zeros([data.shape[0], data.shape[1]*3, data.shape[2]*3, data.shape[3]])
    for i in range(data.shape[0]):
        d1 = data[i,:,:,:]
        d1_expand = expand_data(d1)
        d1_final = preprocess_kernel(d1_expand)
        data_final[i,:,:,:] = d1_final
    print(data_final.shape)
    return data_final

**Wide_and_Deep_CNN_model**

和原論文相比修改了loss function

In [ ]:
import tensorflow as tf
from keras import backend as K
# from tensorflow.keras.utils import np_utils
from keras.layers import Input, concatenate ,Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.models import Model


# Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(K.epsilon()+pt_1)) - K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0 + K.epsilon()))
    return focal_loss_fixed



def Wide_CNN(weeks, days, channel, wide_len, lr=0.005, decay=1e-5,momentum=0.9):
    inputs_deep = Input(shape=(weeks*3, days*3, channel))
    inputs_wide = Input(shape=(wide_len,))

    x_deep = Conv2D(32, (3, 3), strides=(3, 3), padding='same', kernel_initializer='he_normal')(inputs_deep)
    x_deep = MaxPooling2D(pool_size=(3, 3))(x_deep)
    x_deep = Flatten()(x_deep)
    x_deep = Dense(128, activation='relu')(x_deep)

    x_wide = Dense(128, activation='relu')(inputs_wide)

    x = concatenate([x_wide, x_deep])
    x = Dense(64, activation='relu')(x)

    pred = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[inputs_wide, inputs_deep], outputs=pred)

    sgd = tf.keras.optimizers.legacy.SGD(lr=lr, decay=decay, momentum=momentum, nesterov=True)
    model.compile(optimizer=sgd, loss=focal_loss())

    return model


**MAP**

路徑修改成colab用的路徑，並且加上
```self.validation_data = validation_data```

In [ ]:
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
import os

class MyMetric(keras.callbacks.Callback):

    def __init__(self, train_ratio, num, validation_data):
        self.train_ratio = train_ratio
        self.num = num
        self.epoch = 0
        self.validation_data = validation_data


    def precision_at_k(self, r, k):

        assert k >= 1
        r = np.asarray(r)[:k] != 0
        if r.size != k:
            raise ValueError('Relevance score length < k')
        return np.mean(r)

    def average_precision(self, r):

        r = np.asarray(r) != 0
        out = [self.precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
        if not out:
            return 0.
        return np.mean(out)

    def mean_average_precision(self, rs):

        return np.mean([self.average_precision(r) for r in rs])

    def on_epoch_end(self, batch, logs={}):
        self.epoch += 1
        preds = self.model.predict(self.validation_data[0:2])[:,0]
        y = self.validation_data[2][:,0]
        auc = roc_auc_score(y, preds)
#         print(preds.shape, y.shape)

        temp = pd.DataFrame({'label_0':y, 'label_1':1-y, 'preds_0':preds, 'preds_1':1-preds})

        map1 = self.mean_average_precision([list(temp.sort_values(by='preds_0',ascending=0).label_0[:100]),
                                list(temp.sort_values(by='preds_1',ascending=0).label_1[:100])])
        map2 = self.mean_average_precision([list(temp.sort_values(by='preds_0',ascending=0).label_0[:200]),
                                list(temp.sort_values(by='preds_1',ascending=0).label_1[:200])])


        print('AUC:%.4f     MAP@100:%.4f      MAP@200:%.4f  \n'%(auc, map1, map2))

        log = 'Epoch:%2d   AUC:%.4f     MAP@100:%.4f      MAP@200:%.4f  \n'%(self.epoch, auc, map1, map2)
        path = '/content/drive/MyDrive/BDA/train_ratio_%.1f_num_%d.txt'%(self.train_ratio, self.num)
        if os.path.exists(path):
            fp=open(path,"a+",encoding="utf-8")
            fp.write(log + '\n')
        else:
            fp=open(path,"w",encoding="utf-8")
            fp.write(log + '\n')


##建立模型

In [ ]:
# 1. Read data and label
print('Read data and label')
train_data = pd.read_csv('/content/drive/MyDrive/BDA/electricity_theft_final_train.csv')

Read data and label


In [ ]:
from datetime import datetime
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler #正規化

# from function import *
# from keras_metric import *
# from wide_cnn import *

#if __name__ == '__main__':

with tf.device('/device:GPU:0'):

  label = train_data['FLAG'].to_frame()
  data = train_data.drop(['FLAG','CONS_NO'],axis='columns')
  data = data.iloc[:,:1029]


  scaler = MinMaxScaler()
  data_slr = scaler.fit_transform(data.T)
  data_slr = pd.DataFrame(data_slr.T)


  # 2. Split Train dataset and Test dataset with ratio (50%, 60%, 70%, 80%)
  # print('Split Train dataset and Test dataset with ratio (50%, 60%, 70%, 80%)')

  valr = 0.7
  print('Train split ratio:%.2f'%valr)

  X_train_wide, X_test_wide, Y_train, Y_test = train_test_split(data_slr.values, label.values, test_size = 1 - valr, random_state = 2017)

  # 为了符合卷积神经网络模型的输入要求，CNN输入的格式是 [batch_size, height, width, channels]。
  '''
  换成一个四维数组。
  第一维表示样本的数量（与 DataFrame 中的行数相同）。
  第二维设置为 1，表示我们为每个样本保留单个通道。
  第三维通过将 DataFrame 中的列数除以 7 计算得到。这假设每个样本有 7 个特征。
  第四维设置为 7，表示每个样本有 7 个特征。
  再將這四維護換位子，(0,2,3,1)的意思是，把第二維移到第四維的位子
  '''
  X_train_deep = X_train_wide.reshape(X_train_wide.shape[0],1,-1,7).transpose(0,2,3,1)
  X_test_deep = X_test_wide.reshape(X_test_wide.shape[0],1,-1,7).transpose(0,2,3,1)


  weeks, days, channel = X_train_deep.shape[1], X_train_deep.shape[2], 1
  wide_len = X_train_wide.shape[1]

  print(X_train_wide.shape, X_train_deep.shape)
  print(X_test_wide.shape, X_test_deep.shape)







Train split ratio:0.70
(29555, 1029) (29555, 147, 7, 1)
(12667, 1029) (12667, 147, 7, 1)


```  with tf.device('/device:GPU:0'): ``` 用於啟動gpu，若無gpu則砍掉

In [ ]:
  with tf.device('/device:GPU:0'):
    X_train_pre = self_define_cnn_kernel_process(X_train_deep)
    X_test_pre = self_define_cnn_kernel_process(X_test_deep)

(29555, 441, 21, 1)
(12667, 441, 21, 1)


模型參數可嘗試去做設定

In [ ]:
'''
from sklearn.model_selection import GridSearchCV
# 自動條參數，電腦不燒可以試試看
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV
with tf.device('/device:GPU:0'):


'''

In [ ]:
with tf.device('/device:GPU:0'):
# each model run 10 times and get the avg metric result
  for i in range(1):
      print('Round: %d'%i)
      model=Wide_CNN(weeks, days, channel, wide_len)

      if i == 0:
          print(model.summary())

      model.fit([X_train_wide, X_train_pre], Y_train, batch_size=64, epochs=30, verbose=1,
                validation_data=([X_test_wide, X_test_pre], Y_test) , callbacks = [MyMetric(train_ratio=valr, num=i, validation_data=(X_test_wide, X_test_pre, Y_test))])





Round: 0
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 441, 21, 1)]         0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 147, 7, 32)           320       ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 49, 2, 32)            0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 input_2 (InputLayer)        [(None, 1029)]               0         []               

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/30
  2/462 [..............................] - ETA: 1:51 - loss: 0.1767 

396/396 [==============================] - 2s 6ms/step
AUC:0.6378     MAP@100:0.5971      MAP@200:0.5933  

462/462 [==============================] - 15s 23ms/step - loss: 0.0325 - val_loss: 0.0280
Epoch 2/30
396/396 [==============================] - 1s 2ms/step
AUC:0.6928     MAP@100:0.6981      MAP@200:0.7029  

462/462 [==============================] - 5s 10ms/step - loss: 0.0262 - val_loss: 0.0265
Epoch 3/30
396/396 [==============================] - 3s 8ms/step
AUC:0.7149     MAP@100:0.7597      MAP@200:0.7488  

462/462 [==============================] - 8s 16ms/step - loss: 0.0249 - val_loss: 0.0260
Epoch 4/30
396/396 [==============================] - 1s 2ms/step
AUC:0.7409     MAP@100:0.7814      MAP@200:0.7795  

462/462 [==============================] - 6s 13ms/step - loss: 0.0242 - val_loss: 0.0254
Epoch 5/30
396/396 [==============================] - 1s 2ms/step
AUC:0.7538     MAP@100:0.8004      MAP@200:0.7891  

462/462 [==============================] - 5s 11ms/step